
### BUSINESS BROCHURE:

We create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.


In [1]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """
    url: str
    title: str
    body: str
    links: List[str]
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

## Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding.

In [4]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [6]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [7]:
get_links("https://anthropic.com")

{'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://anthropic.com/careers'},
  {'type': 'team page', 'url': 'https://anthropic.com/team'},
  {'type': 'research page', 'url': 'https://anthropic.com/research'}]}

## No lets proceed to make the brochure!

Assemble all the details into another prompt to GPT4-o

In [8]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [ ]:
print(get_all_details("https://anthropic.com"))

In [9]:
# We define a system prompt to make our results display in markdown
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [10]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncate if more than 20,000 characters
    return user_prompt

In [11]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_brochure("Anthropic", "https://anthropic.com")

## A minor recall improvement - Let's stream our results

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [12]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [13]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'Claude page', 'url': 'https://www.anthropic.com/claude'}]}


# Anthropic: Building Safe AI for a Flourishing Future

Welcome to Anthropic! We are an innovative AI safety and research company headquartered in San Francisco. Our mission is to develop reliable, interpretable, and steerable AI systems that empower individuals and organizations while ensuring safety at the frontier of advanced technology.

## Our Purpose
At Anthropic, we recognize the powerful impact AI can have on the world. We are committed to building systems that people can trust and exploring both the opportunities and risks that AI presents.

## Our Work
- **Claude**: Our advanced AI assistant, Claude, supports teams by enhancing collaboration, performing complex cognitive tasks, and driving efficiency across industries.
- **Research**: We conduct pioneering research into AI safety, focusing on areas like interpretability, reinforcement learning, and policy analysis.
- **Collaboration**: We work alongside civil society, government, academia, and industry to promote AI safety and reliability.

## Our Team
Anthropic boasts a diverse and interdisciplinary team of researchers, engineers, policy experts, and business leaders. Our collaborative work environment promotes high trust, where individuals contribute freely and innovatively toward our common goals.

### Values that Define Us
1. **Mission-Driven**: We are dedicated to ensuring that transformative AI helps society thrive.
2. **High Trust Environment**: We prioritize honesty, emotional maturity, and open dialogue.
3. **Collaboration as a Core**: Every member works collectively to push boundaries and innovate.
4. **Pragmatic Solutions**: We celebrate simplicity and efficacy in problem-solving.

## Careers at Anthropic
Join us in shaping the future of safe AI! We are looking for talented individuals from diverse backgrounds in ML, physics, policy, and more.

### What We Offer
- **Comprehensive Benefits**: Health, dental, and vision insurance, along with generous parental leave and mental health support.
- **Flexible Work Environment**: We understand work-life balance and offer flexible time-off policies and remote work options where feasible.
- **Career Growth**: Access to educational stipends, mentorship, and opportunities to work on impactful projects.

### Hiring Process
Our interview process is designed to minimize bias and focus on practical skills, ensuring that we assess candidates fairly regardless of their background in traditional AI or ML work. We welcome applicants from various disciplines, believing in the value of diverse perspectives.

## Join Us
If you're passionate about making AI safe, reliable, and beneficial, we want to hear from you! Explore our open positions and take the first step toward a fulfilling career with Anthropic.

---

### Contact Us
For more information about our products, research, or careers, visit our website at [Anthropic](https://www.anthropic.com) or connect with us on [LinkedIn](https://www.linkedin.com) and [Twitter](https://twitter.com).

Together, let’s harness the power of AI to create a brighter future for all!